In [1]:
from datetime import datetime as dt
from arcgis.gis import GIS
from arcgis.geoanalytics.analyze_patterns import forest, glr
gis = GIS('https://ndhagsb01.esri.com/portal', 
          'admin', 
          'esri.agp2', 
          profile="your_enterprise_portal", verify_cert=False)

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime as dt

import arcgis
import arcgis.geoanalytics
from arcgis.gis import GIS

from arcgis.geoanalytics.manage_data import run_python_script
from arcgis.geoanalytics.analyze_patterns import forest, glr
from arcgis.geoanalytics.summarize_data import describe_dataset

In [3]:
import arcgis
arcgis.env.verbose = True

In [4]:
data = gis.content.search('timeseries_data_17_18_19_1station')[0]

In [5]:
data

<Item title:"timeseries_data_17_18_19_1station291167" type:Table Layer owner:admin>

In [6]:
series_data = data.tables[0]

In [7]:
df = series_data.query(as_df=True)

In [8]:
df

,Barometric_pressure,Carbon_monoxide,Nitric_oxide__NO_,Nitrogen_dioxide__NO2_,OBJECTID,Outdoor_Temperature,Oxides_of_nitrogen__NOx_,Ozone,PM10_Total_0_10um_STP,PM2_5___Local_Conditions,Reactive_oxides_of_nitrogen__NOy_,Relative_Humidity,Sulfur_dioxide,Wind_Direction___Resultant,Wind_Speed___Resultant,datetime,globalid,unique_id
0,992.7,0.059,0.70,3.0,1,86.9,4.0,0.033,23.0,15.7,3.7,50.0,0.8,206.0,9.2,2017-04-28 15:00,{8E2CB27D-96C6-6FA2-46E9-1EA28BDCC1FF},010730023
1,999.0,NaN,0.70,2.7,2,43.9,3.5,0.021,5.0,9.0,3.8,69.0,0.0,285.0,3.7,2017-11-19 07:00,{98B0D3C7-7219-2C73-98C3-31F862DC2E86},010730023
2,995.8,NaN,31.40,21.5,3,66.2,55.9,0.007,50.0,19.0,48.5,69.0,6.4,150.0,2.0,2017-05-26 06:00,{5467D1C2-503A-A94F-11BC-C1DFB93CD0EA},010730023
3,995.2,0.422,8.55,18.9,4,68.0,28.1,NaN,15.0,9.9,24.0,91.0,0.3,241.0,3.5,2017-03-28 07:00,{27FC92CA-4941-ADB0-7E5E-EFB9018E02BB},010730023
4,978.2,NaN,0.25,4.9,5,73.6,5.3,0.027,NaN,12.9,4.4,70.0,0.2,216.0,7.6,2018-03-19 18:00,{37EA6335-6768-0C0B-D0D5-56D9A3F6C6BA},010730023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18226,997.4,0.285,0.00,10.1,21983,59.5,10.2,0.030,10.0,12.8,NaN,72.0,NaN,216.0,2.6,2017-11-01 03:00,{0B3F91C9-6613-6609-998B-7E5446092E3C},010730023
18227,1009.1,NaN,0.95,7.9,22016,42.1,9.1,0.011,NaN,12.6,9.3,93.0,0.0,104.0,3.6,2018-02-12 17:00,{B691D558-2F6C-B180-1C61-4A1280765D7E},010730023
18228,995.2,NaN,20.10,NaN,22073,69.4,NaN,0.002,24.0,15.8,33.7,85.0,NaN,143.0,1.2,2017-09-28 06:00,{33BDA1B1-87E8-B4DA-F57E-87BB9AD05FE9},010730023
18229,997.2,0.351,0.10,4.7,22273,68.5,4.9,NaN,NaN,NaN,4.1,80.0,NaN,176.0,5.5,2017-02-28 19:00,{732EA0C0-B98A-B2A6-383B-378A608AE4E5},010730023


In [52]:
def code():
    from pyspark.sql.functions import pandas_udf, PandasUDFType
    from pyspark.sql.functions import concat, col, lit
    from datetime import datetime as dt
    from pyspark.sql import SQLContext
    df = layers[0]
    df = df.withColumn('flag', lit(1))
    @pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
    def addition_test(pdf):
        pdf['Nitric_oxide__NO_'] = pdf['Nitric_oxide__NO_'] + 1
        return pdf    
    df = df.groupby(['flag']).apply(addition_test)
    df.write.format("webgis").save("addition_test" + str(dt.now().microsecond))
    

In [54]:
gis.content.search('addition_test')[0]

<Item title:"addition_test224637" type:Table Layer owner:admin>

In [61]:
def code():
    from pyspark.sql.functions import pandas_udf, PandasUDFType
    from pyspark.sql.functions import concat, col, lit
    from datetime import datetime as dt
    from pyspark.sql import SQLContext
    df = layers[0]
    cols = ['Outdoor_Temperature', 'Ozone', 'PM10_Total_0_10um_STP',
    'PM2_5___Local_Conditions', 'Relative_Humidity',
    'Wind_Direction___Resultant',
    'Wind_Speed___Resultant', 'datetime']
    df = df.select(cols)
    df = df.withColumn('flag', lit(1))
    
    from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, FloatType, TimestampType, StringType

    schema = StructType([StructField('Date', TimestampType(), True), 
                         StructField('Outdoor_Temperature', FloatType(), True),
                         StructField('Ozone', FloatType(), True),
                         StructField('PM10_Total_0_10um_STP', FloatType(), True),
                         StructField('PM2_5___Local_Conditions', FloatType(), True),
                         StructField('Relative_Humidity', FloatType(), True),
                         StructField('Wind_Direction___Resultant', FloatType(), True),
                         StructField('Wind_Speed___Resultant', FloatType(), True),
                         StructField('datetime', StringType(), True),
                         StructField('flag', IntegerType(), True)])
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def interpolate_code(pdf):  
        pdf['Date'] = pdf['datetime'].astype('datetime64[ns]')
        pdf = pdf.set_index('Date')
        pdf.sort_values(by=['PM2_5___Local_Conditions'], inplace=True)
        pdf = pdf.interpolate('nearest')
        pdf['PM2_5___Local_Conditions'] = pdf['PM2_5___Local_Conditions'] + 1
        pdf.reset_index(inplace=True)                
        return pdf

    df = df.groupby(['flag']).apply(interpolate_code)
    df.write.format("webgis").save("interpolated_data" + str(dt.now().microsecond))

In [62]:
run_python_script(code=code, layers=[series_data])

Submitted.
Executing...
Executing (RunPythonScript): RunPythonScript "def code():\n    from pyspark.sql.functions import pandas_udf, PandasUDFType\n    from pyspark.sql.functions import concat, col, lit\n    from datetime import datetime as dt\n    from pyspark.sql import SQLContext\n    df = layers[0]\n    cols = ['Outdoor_Temperature', 'Ozone', 'PM10_Total_0_10um_STP',\n    'PM2_5___Local_Conditions', 'Relative_Humidity',\n    'Wind_Direction___Resultant',\n    'Wind_Speed___Resultant', 'datetime']\n    df = df.select(cols)\n    df = df.withColumn('flag', lit(1))\n    \n    from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, FloatType, TimestampType, StringType\n\n    schema = StructType([StructField('Date', TimestampType(), True), \n                         StructField('Outdoor_Temperature', FloatType(), True),\n                         StructField('Ozone', FloatType(), True),\n                         StructField('PM10_Total_0_10um_STP', FloatType(), Tru

[{'type': 'esriJobMessageTypeInformative',
  'description': 'Executing (RunPythonScript): RunPythonScript "def code():\\n    from pyspark.sql.functions import pandas_udf, PandasUDFType\\n    from pyspark.sql.functions import concat, col, lit\\n    from datetime import datetime as dt\\n    from pyspark.sql import SQLContext\\n    df = layers[0]\\n    cols = [\'Outdoor_Temperature\', \'Ozone\', \'PM10_Total_0_10um_STP\',\\n    \'PM2_5___Local_Conditions\', \'Relative_Humidity\',\\n    \'Wind_Direction___Resultant\',\\n    \'Wind_Speed___Resultant\', \'datetime\']\\n    df = df.select(cols)\\n    df = df.withColumn(\'flag\', lit(1))\\n    \\n    from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, FloatType, TimestampType, StringType\\n\\n    schema = StructType([StructField(\'Date\', TimestampType(), True), \\n                         StructField(\'Outdoor_Temperature\', FloatType(), True),\\n                         StructField(\'Ozone\', FloatType(), True),\\

In [63]:
gis.content.search('interpolated_data')[0]

<Item title:"interpolated_data476415" type:Table Layer owner:admin>